In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


wm_station_data에서 사용한 함수들

In [ ]:
#컬럼별 데이터프레임을 입력받아 membership_func에 넣어서 각 값마다의 퍼지 소속도를 데이터 프레임으로 출력
#해당 소속도는 아래의 함수 membership_fun을 통해 계산
#binary 변수의 경우 def fuzzy_set_maker_bi 참고
def fuzzy_set_maker(df,var_name, centroid_list):

  df_to_list = df.values.tolist()
  fuzzy_list = []
  col = []
  centroid_list.sort()

  for i in range(len(centroid_list)):
    col_name = "Rank" + str(i+1)
    col.append(col_name)
  col.insert(0, var_name)

  for i in range(len(df_to_list)):
    fuzzy_set = membership_func(df_to_list[i], centroid_list)
    fuzzy_list.append(fuzzy_set)
  
  fuzzy_df = pd.DataFrame(fuzzy_list, columns = col)

  return fuzzy_df

In [ ]:
# 중심값 리스트와 값을 넣으면 자동으로 퍼지 소속도 계산해주는 함수
# value는 관측값, centroid_list는 해당 컬럼의 중심값들을 담은 리스트

def membership_func(value, centroid_list):
  fuzzy_set = [0] * len(centroid_list)

  # 관측값이 가장 작은 중심값보다 작거나 가장 큰 중심값보다 클 때 적용
  if value <= centroid_list[0]:
    fuzzy_set[0] = 1
  elif value >= centroid_list[-1]:
    fuzzy_set[-1] = 1


  for i in range(len(centroid_list)):
    if i != len(centroid_list) - 1:
      if value > centroid_list[i] and value <= centroid_list[i+1]:
        small_tri_base = value - centroid_list[i]
        big_tri_base = centroid_list[i+1] - centroid_list[i]
        fuzzy_set[i] = small_tri_base/big_tri_base
        fuzzy_set[i+1] = 1 - fuzzy_set[i]
  

  fuzzy_set.insert(0, value)
  return fuzzy_set

In [ ]:
# 관측값이 연속값이 아니라 binary 변수일 경우

def fuzzy_set_maker_bi(df,var_name):

  df_to_list = df.values.tolist()
  fuzzy_list = []
  col = []

  for i in range(2):
    col_name = "Rank" + str(i+1)
    col.append(col_name)
  col.insert(0, var_name)

  for i in range(len(df_to_list)):
    if df_to_list[i] == 0:
      fuzzy_set = [1, 0]
    else:
      fuzzy_set = [0, 1]
    fuzzy_set.insert(0, df_to_list[i])
    fuzzy_list.append(fuzzy_set)
  
  fuzzy_df = pd.DataFrame(fuzzy_list, columns = col)

  return fuzzy_df

In [ ]:
explanatory_col_list = ['LANTERN_STATUS', 'MAIN_VOLT_STATUS','BATTERY_VOLT_STATUS', 'LANTERN_CURR_STATUS', 'CHARGER_CURR_STATUS', 'DISCHARGER_CURR_STATUS']

In [ ]:
target_col = ['SOLAR_VOLT_STATUS']

In [ ]:
def rule_maker(explan_df_list, target_df):

  #explanatory_col_list = ['LANTERN_STATUS', 'MAIN_VOLT_STATUS','BATTERY_VOLT_STATUS', 'LANTERN_CURR_STATUS', 'CHARGER_CURR_STATUS', 'DISCHARGER_CURR_STATUS']
  
  explan_list = []
  explan_fuzzy = []
  for df in explan_df_list:
    temp_list = df.values.tolist()
    explan_list.append(temp_list)
    temp_list = df.columns.tolist()
    explan_fuzzy.append(temp_list)

  target_list = target_df.values.tolist()
  target_fuzzy = target_df.columns.tolist()

  rule_dic = {}

  for i in range(len(target_list)):
    w = 0
    key = ""
    for j in range(len(explan_list)):
      max_value = max(explan_list[j][i][1:])
      max_value_fuzzy = explan_fuzzy[j][explan_list[j][i][1:].index(max_value) + 1]
      w += max_value
      key = key + explan_fuzzy[j][0] + "['" + max_value_fuzzy + "' ] & "
    
    max_target = max(target_list[i][1:])
    max_target_fuzzy = target_fuzzy[target_list[i][1:].index(max_target)+ 1]
    y = target_list[i][0]


    key = key + target_fuzzy[0] + "['" + max_target_fuzzy + "']"
   
    temp_list = [w, y, max_target_fuzzy]

    if key in rule_dic:
      rule_dic[key].append(temp_list)
    else:
      rule_dic[key] = []
      rule_dic[key].append(temp_list)

  

  return rule_dic

In [ ]:
 def target_membership(fuzzy,value, target_centroid):
   
   fuzzy_set = 0
   centroid = target_centroid[fuzzy-1]
   if fuzzy != 1 and fuzzy != len(target_centroid):
     left = target_centroid[fuzzy-2]
     right = target_centroid[fuzzy]
     if value < left and value > right:
       fuzzy_set = 0
     elif value < centroid:
       fuzzy_set = (value - left)/(centroid - left)
     elif value >= centroid:
        fuzzy_set = (right - value)/(right - centroid)
   elif fuzzy == 1:
     if value > target_centroid[fuzzy]:
       fuzzy_set = 0
     elif value > target_centroid[0]:
       fuzzy_set = (target_centroid[fuzzy] - value)/(target_centroid[fuzzy] - centroid)
     else:
       fuzzy_set = 1
   elif fuzzy == len(target_centroid):
     if value < target_centroid[fuzzy-2]:
       fuzzy_set = 0
     elif value < target_centroid[-1]:
       fuzzy_set = (value - target_centroid[fuzzy-2])/(centroid - target_centroid[fuzzy-2])
     else:
       fuzzy_set = 1
   return fuzzy_set

In [ ]:
def rule_doc(rule_dic, target_centroid):
  key_list = list(rule_dic.keys())
  rule_list = []

  for key in key_list:
    temp_list = rule_dic[key]
    weight_value_sum = 0
    weight_sum = 0
    value_minus_avg = 0
    max_y = 0
    min_y = 0

    for i in range(len(temp_list)):
      weight_value_sum += temp_list[i][0] * temp_list[i][1]
      weight_sum += temp_list[i][0]

      if max_y < temp_list[i][1]:
        max_y = temp_list[i][1]
      
      if min_y > temp_list[i][1]:
        min_y = temp_list[i][1]
    
    av = weight_value_sum/weight_sum
    for i in range(len(temp_list)):
      value_minus_avg += abs(temp_list[i][1]-av) * temp_list[i][0]
    var = value_minus_avg/weight_sum

    target_mebership = str(temp_list[i][2])
    target_mebership = int(target_mebership[-1])
    

    if min_y == max_y:
      doc = 1
    else:  
      doc = (1 - var/abs(max_y - min_y))*target_membership(target_mebership, av, target_centroid)

    doc_list = [key, doc]
    rule_list.append(doc_list)
  
  rule_df = pd.DataFrame(rule_list, columns = ['rule', 'doc'])
 


  return rule_df

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/fuzzy/fuzzy_master_data/TB_STATUS_1040111.csv')
use_col = ['LANTERN_STATUS', 'MAIN_VOLT_STATUS','BATTERY_VOLT_STATUS', 'LANTERN_CURR_STATUS', 'CHARGER_CURR_STATUS', 'DISCHARGER_CURR_STATUS', 'SOLAR_VOLT_STATUS']
df = df[use_col]
df = df.fillna(0)
df

,LANTERN_STATUS,MAIN_VOLT_STATUS,BATTERY_VOLT_STATUS,LANTERN_CURR_STATUS,CHARGER_CURR_STATUS,DISCHARGER_CURR_STATUS,SOLAR_VOLT_STATUS
0,0.0,13.5,13.7,0.00,0.18,0.00,14.0
1,0.0,13.5,13.7,0.00,0.19,0.00,14.0
2,0.0,13.5,13.7,0.00,0.20,0.00,14.0
3,0.0,13.5,13.7,0.00,0.18,0.00,14.0
4,0.0,13.5,13.7,0.00,0.15,0.00,14.0
...,...,...,...,...,...,...,...
1080175,1.0,12.4,12.7,0.29,0.00,0.55,0.0
1080176,1.0,12.4,12.7,0.29,0.00,0.55,0.0
1080177,1.0,12.4,12.7,0.29,0.00,0.55,0.0
1080178,1.0,12.4,12.7,0.29,0.00,0.55,0.0


In [ ]:
main_centroid = [12.47007145, 13.35150439, 14.57403326]
main_fuzzy_set = fuzzy_set_maker(df['MAIN_VOLT_STATUS'], 'main', main_centroid)

In [ ]:
lan_fuzzy_set = fuzzy_set_maker_bi(df['LANTERN_STATUS'], 'lan')

In [ ]:
battery_centroid = [13.2668709 , 14.77602876, 12.71586477, 13.82805277]
battery_fuzzy_set = fuzzy_set_maker(df['BATTERY_VOLT_STATUS'], 'battery', battery_centroid)

In [ ]:
lancurr_centroid = [2.24025549e-04, 2.83144673e-01]
lancurr_fuzzy_set = fuzzy_set_maker(df['LANTERN_CURR_STATUS'], 'lancurr', lancurr_centroid)

In [ ]:
charger_centroid = [0.85008603, 1.84222279, 0.01976889, 3.51127932]
charger_fuzzy_set = fuzzy_set_maker(df['CHARGER_CURR_STATUS'], 'charger', charger_centroid)

In [ ]:
discharger_centroid = [0.00298113, 0.55060921, 0.26205022]
discharger_fuzzy_set = fuzzy_set_maker(df['DISCHARGER_CURR_STATUS'], 'discharger', discharger_centroid)

In [ ]:
solar_centroid = [0.08583593, 6.47214301, 13.80720279, 20.19871758]
solar_fuzzy_set = fuzzy_set_maker(df['SOLAR_VOLT_STATUS'], 'solar', solar_centroid)

In [ ]:
solar_fuzzy_set

,solar,Rank1,Rank2,Rank3,Rank4
0,14.0,0.0,0.0,0.030165,0.969835
1,14.0,0.0,0.0,0.030165,0.969835
2,14.0,0.0,0.0,0.030165,0.969835
3,14.0,0.0,0.0,0.030165,0.969835
4,14.0,0.0,0.0,0.030165,0.969835
...,...,...,...,...,...
1080175,0.0,1.0,0.0,0.000000,0.000000
1080176,0.0,1.0,0.0,0.000000,0.000000
1080177,0.0,1.0,0.0,0.000000,0.000000
1080178,0.0,1.0,0.0,0.000000,0.000000


In [ ]:
explan_df_list = [main_fuzzy_set, battery_fuzzy_set, lancurr_fuzzy_set, lan_fuzzy_set, charger_fuzzy_set, discharger_fuzzy_set]

In [ ]:
rule_dic = rule_maker(explan_df_list, solar_fuzzy_set)
key_list = list(rule_dic.keys())

In [ ]:
rule_df = rule_doc(rule_dic, solar_centroid)

In [ ]:
rule_df

,rule,doc
0,main['Rank3' ] & battery['Rank2' ] & lancurr['...,0.033141
1,main['Rank3' ] & battery['Rank2' ] & lancurr['...,0.043245
2,main['Rank3' ] & battery['Rank2' ] & lancurr['...,0.017658
3,main['Rank3' ] & battery['Rank2' ] & lancurr['...,0.000982
4,main['Rank1' ] & battery['Rank2' ] & lancurr['...,0.000982
...,...,...
194,main['Rank3' ] & battery['Rank4' ] & lancurr['...,0.045810
195,main['Rank1' ] & battery['Rank2' ] & lancurr['...,0.480958
196,main['Rank1' ] & battery['Rank1' ] & lancurr['...,0.003798
197,main['Rank1' ] & battery['Rank1' ] & lancurr['...,0.017431


In [ ]:
rule_df.to_csv("/content/drive/MyDrive/Colab Notebooks/fuzzy/fuzzy_master_data/master_rules.csv")

# 새 섹션